In [146]:
import pandas as pd
from pandas.core.frame import DataFrame
import numpy as np
import seaborn as sns

filename = 'healthcare-dataset-stroke-data.csv'
cols = ['gender','age','hypertension','heart_disease','ever_married', 'work_type', 'Residence_type','avg_glucose_level','bmi', 'smoking_status','stroke']

df = pd.read_csv(filename, usecols=cols)

In [147]:
# Check datatype each column of dataframe
cat_df = df.select_dtypes(include=['object'])
num_df = df.select_dtypes(exclude=['object'])

def printColumnTypes(non_numeric_df: DataFrame, numeric_df: DataFrame):
    '''separates non-numeric and numeric columns'''
    print("Non-Numeric columns:")
    for col in non_numeric_df:
        print(f"{col}")
    print("")
    print("Numeric columns:")
    for col in numeric_df:
        print(f"{col}")

printColumnTypes(cat_df, num_df)

df.info()

Non-Numeric columns:
gender
ever_married
work_type
Residence_type
smoking_status

Numeric columns:
age
hypertension
heart_disease
avg_glucose_level
bmi
stroke
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5110 entries, 0 to 5109
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   gender             5110 non-null   object 
 1   age                5110 non-null   float64
 2   hypertension       5110 non-null   int64  
 3   heart_disease      5110 non-null   int64  
 4   ever_married       5110 non-null   object 
 5   work_type          5110 non-null   object 
 6   Residence_type     5110 non-null   object 
 7   avg_glucose_level  5110 non-null   float64
 8   bmi                4909 non-null   float64
 9   smoking_status     5110 non-null   object 
 10  stroke             5110 non-null   int64  
dtypes: float64(3), int64(3), object(5)
memory usage: 439.3+ KB


In [148]:
def printUniqueValue(cols: list):
    for col in cols:
        print(f"{col}: {df[col].unique()}")
        
non_numeric_col = ['gender','ever_married','work_type','Residence_type', 'smoking_status']

printUniqueValue(non_numeric_col)

gender: ['Male' 'Female' 'Other']
ever_married: ['Yes' 'No']
work_type: ['Private' 'Self-employed' 'Govt_job' 'children' 'Never_worked']
Residence_type: ['Urban' 'Rural']
smoking_status: ['formerly smoked' 'never smoked' 'smokes' 'Unknown']


In [149]:
# Data cleansing
num_gender = { 'Female': 0, 'Male': 1 }
num_ever_married = { 'No': 0, 'Yes': 1 }
num_smoking_status = {
    'formerly smoked': 0,
    'never smoked': 1,
    'smokes': 2,
    'Unknown': 3,
}
num_work_type = {
    'children': 0,
    'Govt_job': 1,
    'Never_worked': 2,
    'Private': 3,
    'Self-employed': 4,
}
num_residence_type = {
    'Urban': 0,
    'Rural': 1
}

# Remove Other value in gender column
df = df[df['gender'] != 'Other']

# Label Encoding
df['gender'] = df['gender'].replace(num_gender)
df['ever_married'] = df['ever_married'].replace(num_ever_married)
df['Residence_type'] = df['Residence_type'].replace(num_residence_type)
df['smoking_status'] = df['smoking_status'].replace(num_smoking_status)
df['work_type'] = df['work_type'].replace(num_work_type)

mean_bmi_replacement_value = df.loc[:,'bmi'].dropna().mean()

df['bmi'] = df.loc[:, 'bmi'].fillna(mean_bmi_replacement_value)

In [150]:
df.head()

,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,1,67.0,0,1,1,3,0,228.69,36.60000,0,1
1,0,61.0,0,0,1,4,1,202.21,28.89456,1,1
2,1,80.0,0,1,1,3,1,105.92,32.50000,1,1
3,0,49.0,0,0,1,3,0,171.23,34.40000,2,1
4,0,79.0,1,0,1,4,1,174.12,24.00000,1,1


In [151]:

# sns.heatmap(data=df.corr(), annot=True)

In [152]:
print(df['stroke'].value_counts() / len(df))

0    0.951262
1    0.048738
Name: stroke, dtype: float64


In [153]:
# x = np.array(df.loc[:, df.columns != 'stroke'])
# y = np.array(df.loc[:, df.columns == 'stroke']).reshape(-1, 1)

X = df.iloc[:, :-1].values
y = df.iloc[:, 10].values

In [154]:
df.columns

Index(['gender', 'age', 'hypertension', 'heart_disease', 'ever_married',
       'work_type', 'Residence_type', 'avg_glucose_level', 'bmi',
       'smoking_status', 'stroke'],
      dtype='object')

In [155]:
from sklearn.preprocessing import MinMaxScaler

mms = MinMaxScaler()
mms.fit(X)

mms_np = mms.transform(X)

In [156]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(mms_np,y, test_size=0.3, random_state=5)

In [157]:
from sklearn.neighbors import KNeighborsClassifier
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import GridSearchCV, StratifiedKFold,  KFold
from sklearn.metrics import confusion_matrix

pipeline = Pipeline(steps = [['smote', SMOTE(random_state=11)],
                                ['scaler', MinMaxScaler()],
                                ['classifier', KNeighborsClassifier(weights='distance',metric='euclidean')]])

stratified_kfold = StratifiedKFold(n_splits=10,
                                       shuffle=True,
                                       random_state=11)

param_grid = {'classifier__n_neighbors': [1, 3, 5, 7, 9, 11, 13, 15, 17, 19]}
grid_search = GridSearchCV(estimator=pipeline,
                           param_grid=param_grid,
                           scoring='accuracy',
                           cv=stratified_kfold,
                           n_jobs=-1)


grid_search.fit(X_train, y_train)
cv_score = grid_search.best_score_
test_score = grid_search.score(X_test, y_test)
grid_search_df = pd.DataFrame(grid_search.cv_results_)

print(f'Cross-validation score: {cv_score}\nTest score: {test_score}')
grid_search_df

Cross-validation score: 0.881999280158991
Test score: 0.8773646444879322


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_classifier__n_neighbors,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,split5_test_score,split6_test_score,split7_test_score,split8_test_score,split9_test_score,mean_test_score,std_test_score,rank_test_score
0,0.075729,0.021246,0.066216,0.024873,1,{'classifier__n_neighbors': 1},0.893855,0.868715,0.860335,0.860335,0.907821,0.871508,0.876751,0.896359,0.876751,0.907563,0.881999,0.017168,1
1,0.053118,0.006192,0.051069,0.007281,3,{'classifier__n_neighbors': 3},0.860335,0.843575,0.826816,0.821229,0.885475,0.835196,0.817927,0.873950,0.829132,0.854342,0.844798,0.021856,2
2,0.053318,0.009641,0.059317,0.017663,5,{'classifier__n_neighbors': 5},0.835196,0.824022,0.815642,0.801676,0.871508,0.824022,0.789916,0.857143,0.809524,0.845938,0.827459,0.024046,3
3,0.054363,0.011952,0.065376,0.031572,7,{'classifier__n_neighbors': 7},0.821229,0.807263,0.796089,0.790503,0.849162,0.815642,0.778711,0.829132,0.773109,0.823529,0.808437,0.022666,4
4,0.070021,0.029008,0.073480,0.024622,9,{'classifier__n_neighbors': 9},0.807263,0.798883,0.779330,0.779330,0.835196,0.804469,0.773109,0.820728,0.767507,0.812325,0.797814,0.021166,5
5,0.081125,0.032111,0.074224,0.015868,11,{'classifier__n_neighbors': 11},0.787709,0.784916,0.768156,0.768156,0.815642,0.796089,0.753501,0.809524,0.759104,0.817927,0.786073,0.022302,6
6,0.060163,0.018334,0.061721,0.005754,13,{'classifier__n_neighbors': 13},0.782123,0.776536,0.759777,0.748603,0.810056,0.784916,0.747899,0.789916,0.747899,0.812325,0.776005,0.023202,7
7,0.053877,0.011275,0.065427,0.013667,15,{'classifier__n_neighbors': 15},0.770950,0.768156,0.751397,0.754190,0.804469,0.782123,0.750700,0.795518,0.739496,0.812325,0.772932,0.023654,8
8,0.053415,0.011905,0.064716,0.006289,17,{'classifier__n_neighbors': 17},0.773743,0.768156,0.740223,0.754190,0.790503,0.762570,0.745098,0.787115,0.742297,0.806723,0.767062,0.021383,9
9,0.047117,0.004386,0.063069,0.005071,19,{'classifier__n_neighbors': 19},0.762570,0.759777,0.731844,0.751397,0.782123,0.759777,0.742297,0.775910,0.736695,0.803922,0.760631,0.020971,10


In [158]:


model = KNeighborsClassifier(weights='distance',metric='euclidean')

# param_grid = {'n_neighbors': [1, 3, 5, 7, 9, 11, 13, 15]}

# # Define grid search
# grid_search = GridSearchCV(model, param_grid=param_grid, cv=10)

# # Fit grid search to training data
# grid_search.fit(X_train, y_train)

# # Get best estimator and evaluate on test data
# best_knn = grid_search.best_estimator_
# y_pred = best_knn.predict(X_test)

# confusion_matrix(y_test,y_pred)

# accuracy = best_knn.score(X_test, y_test)

# print("Accuracy:", accuracy)
# print("best knn:", best_knn)

# grid_search_df = pd.DataFrame(grid_search.cv_results_)
# grid_search_df

In [159]:
# Features Importants
# from matplotlib import pyplot

# importance = model.feat
# # summarize feature importance
# for i,v in enumerate(importance):
#  print('Feature: %0d, Score: %.5f' % (i,v))
# # plot feature importance
# pyplot.bar([x for x in range(len(importance))], importance)
# pyplot.show()

NotFittedError: This KNeighborsClassifier instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.

In [ ]:
from sklearn import metrics

print("Accuracy:", metrics.accuracy_score(y_test,y_pred))

Accuracy: 0.821917808219178
